In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import re

from sklearn.model_selection import train_test_split #for split the data
from sklearn.metrics import accuracy_score  #for accuracy_score
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.model_selection import GridSearchCV # for Hyper parameter tuning
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn import metrics

import xgboost as xgb
from xgboost import XGBClassifier

UST2 = pd.read_csv('../final-1.csv')
print(UST2.head())
print(UST2.shape)

UST = pd.read_csv('../final-1.csv')
print(UST.head(10))
print(UST.shape)

# See the column data types and non-missing values
print(UST.info())
print(UST.describe())

# seems fine
print(UST['CLI'].mean(), UST['CLI'].median())

print(UST.columns)
print(UST.shape)

all_data = UST

traindf=UST

traindf = pd.get_dummies(traindf, columns = ["term","grade","sub_grade","emp_length","home_ownership","verification_status","loan_status","earliest_cr_line","last_credit_pull_d"],
                             prefix=["Owner","Account","Stg","SvcLn","StgNum","Acctype","Billngtype","UnSolProp","DelType"])

print(traindf.head())

# total of 100 records in dataset
# Divide the data set into two- 70% for train and 30% for test

train, test = train_test_split(traindf, test_size=0.3)

# Check the train test data set shape
train.shape, test.shape

# Add binary label column, modify test
trainBinary = []
testBinary = []

for num in np.random.randint(2, size=train.shape[0]):
    trainBinary.append(num)

for num in np.random.randint(2, size=test.shape[0]):
    testBinary.append(num)

train['binary'] = trainBinary
test['binary'] = testBinary

test.drop(['CLI'], axis = 1, inplace = True)

print(list(train.columns).index('binary'))
print(list(test.columns).index('binary'))

# Save to csv files for XGBoost
train.to_csv('xg_train_final.csv')
test.to_csv('xg_test_final.csv')

# somewhat unnecessary check
print(list(traindf.columns).index('CLI'))


'''
XGBoost
'''
# put traindf into clean.csv - only needs to be run once
# traindf.to_csv('Datasets/clean.csv')

# https://xgboost.readthedocs.io/en/stable/python/python_intro.html#setting-parameters

# get feature names
featureNames = []
for feature in traindf.columns:
    featureNames.append(feature)
# print(featureNames)

# Read data
# data = traindf # pd.DataFrame(np.arange(12).reshape((4,3)), columns=['a', 'b', 'c'])
# label = pd.DataFrame(featureNames, columns = ['label']) # pd.DataFrame(np.random.randint(2, size=traindf.shape[0]))
# dtrain = xgb.DMatrix(data, label=label, enable_categorical=True)

# Read data
# label_column specifies the index of the column containing the true label
dtrain = xgb.DMatrix('xg_train_final.csv?format=csv&label_column=185') # should have cli column
dtest = xgb.DMatrix('xg_test_final.csv?format=csv&label_column=184') # should not have cli column?

# Set parameters
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

# Training
num_round = 10
evallist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(param, dtrain, num_round, evallist)

# Save model
bst.save_model('0001.model')

# Load saved model
# bst = xgb.Booster({'nthread': 4})  # init model
# bst.load_model('model.bin')  # load data

# Prediction
# 7 entities, each contains 10 features
data = np.random.rand(7, 10)
dtest = xgb.DMatrix(data)
ypred = bst.predict(dtest)

# Graph
xgb.plot_importance(bst)


'''
XGBoost Prediction

Don't think I'll be able to get this to work. The error, 'ValueError: Invalid classes inferred from unique values of `y`.'
doesn't have a solution I could find.
'''



# get feature names
featureNames = []
for feature in traindf.columns:
    featureNames.append(feature)

all_features = traindf.drop("CLI",axis=1)
Targeted_feature = traindf["CLI"]

# total of 100 records in dataset
# Divide the data set into two- 70% for train and 30% for test

X_train,X_test,y_train,y_test = train_test_split(all_features, Targeted_feature, test_size=0.3, random_state=42)

# Check the train test data set shape
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
print(y_train)

# fit model
clf = XGBClassifier()
clf.fit(X_train, y_train)

# make predictions for test data
y_pred = clf.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


        id  member_id    CLI        term  int_rate  installment grade  \
0  1077501    1296599   5000   36 months     10.65       162.87     B   
1  1077430    1314167   2500   60 months     15.27        59.83     C   
2  1077175    1313524   2400   36 months     15.96        84.33     C   
3  1076863    1277178  10000   36 months     13.49       339.31     C   
4  1075358    1311748   3000   60 months     12.69        67.79     B   

  sub_grade emp_length home_ownership  ...  out_prncp_inv   total_pymnt  \
0        B2  10+ years           RENT  ...            0.0   5861.071414   
1        C4   < 1 year           RENT  ...            0.0   1008.710000   
2        C5  10+ years           RENT  ...            0.0   3003.653644   
3        C1  10+ years           RENT  ...            0.0  12226.302210   
4        B5     1 year           RENT  ...          766.9   3242.170000   

  total_pymnt_inv  total_rec_prncp  total_rec_int total_rec_late_fee  \
0         5831.78          5000.00    

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '